In [35]:
# Cell 1
import requests
import pandas as pd
from datetime import date
from dateutil import parser, tz
from IPython.display import display

def fetch_json(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()


In [43]:
# Cell 2
today = date.today().isoformat()
sched = fetch_json(f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}")

if not sched.get("dates"):
    print(f"No games scheduled for {today}.")
else:
    for game in sched["dates"][0]["games"]:
        pk          = game["gamePk"]
        away        = game["teams"]["away"]["team"]["name"]
        away_score  = game["teams"]["away"]["score"]
        home        = game["teams"]["home"]["team"]["name"]
        home_score  = game["teams"]["home"]["score"]
        status      = game["status"]["detailedState"]
        print(f"{pk}: {away} {away_score} @ {home} {home_score} — {status}")


777326: Athletics 7 @ New York Yankees 0 — Final
777324: Minnesota Twins 5 @ Detroit Tigers 10 — Final
777325: New York Mets 2 @ Pittsburgh Pirates 9 — Final
777330: Tampa Bay Rays 11 @ Baltimore Orioles 3 — Final
777322: Seattle Mariners 2 @ Texas Rangers 3 — Final
777319: Los Angeles Dodgers 5 @ Kansas City Royals 9 — Final
777328: San Diego Padres 6 @ Cincinnati Reds 4 — Final
777329: St. Louis Cardinals 9 @ Cleveland Guardians 6 — Final
777323: Toronto Blue Jays 1 @ Boston Red Sox 15 — Final
777327: San Francisco Giants 0 @ Chicago White Sox 1 — Final
777318: Miami Marlins 8 @ Arizona Diamondbacks 7 — Final
777321: Colorado Rockies 0 @ Milwaukee Brewers 5 — Final
777320: Chicago Cubs 12 @ Houston Astros 3 — Final
777333: Philadelphia Phillies 1 @ Atlanta Braves 6 — Final
777315: Washington Nationals 2 @ Los Angeles Angels 7 — In Progress


In [44]:
# Cell 3: enter a gamePk and display detailed stats with robust fallback

game_pk = int(input("Enter gamePk: "))

def show_linescore(df, label):
    print(f"{label} Linescore"); display(df)

def show_stats(rows, label):
    print(label); display(pd.DataFrame(rows))

# 1) Try live feed
resp = requests.get(f"https://statsapi.mlb.com/api/v1/game/{game_pk}/feed/live")
if resp.status_code == 200 and "liveData" in (payload := resp.json()):
    feed = payload["liveData"]
    
    # Live linescore
    ls = feed["linescore"]
    df_ls = (
        pd.DataFrame(ls["innings"])[["num","away","home"]]
          .rename(columns={"num":"Inning","away":"Away","home":"Home"})
    )
    df_ls["Away Total"] = ls["teams"]["away"]["runs"]
    df_ls["Home Total"] = ls["teams"]["home"]["runs"]
    show_linescore(df_ls, "Live")

    # Live batting
    bat_live = []
    for side in ("home","away"):
        team = feed["boxscore"]["teams"][side]
        for pid in team.get("batters",[]):
            p = feed["boxscore"]["players"].get(f"ID{pid}", {})
            stats = p.get("stats",{}).get("batting",{})
            bat_live.append({
                "Team": team["team"]["name"],
                "Player": p.get("person",{}).get("fullName",""),
                **{k:v for k,v in stats.items() if isinstance(v,(int,float,str))}
            })
    show_stats(bat_live, "Live Batting")

    # Live pitching
    pit_live = []
    for side in ("home","away"):
        team = feed["boxscore"]["teams"][side]
        for pid in team.get("pitchers",[]):
            p = feed["boxscore"]["players"].get(f"ID{pid}", {})
            stats = p.get("stats",{}).get("pitching",{})
            pit_live.append({
                "Team": team["team"]["name"],
                "Player": p.get("person",{}).get("fullName",""),
                **{k:v for k,v in stats.items() if isinstance(v,(int,float,str))}
            })
    show_stats(pit_live, "Live Pitching")

    # Live play-by-play
    plays = []
    for play in feed["playByPlay"]["allPlays"]:
        a    = play["about"]
        desc = play["result"].get("description","")
        t    = parser.isoparse(a["startTime"]).astimezone(tz.tzlocal()).strftime("%I:%M %p")
        plays.append({
            "Inning": a["inning"],
            "Half":   a["halfInning"],
            "Time":   t,
            "Desc":   desc
        })
    show_stats(plays, f"Live Play-by-Play ({len(plays)} events)")

else:
    # FALLBACK: linescore & boxscore endpoints
    print("Live data unavailable—using boxscore & linescore endpoints.\n")
    
    # Linescore fallback
    ls_fb = requests.get(f"https://statsapi.mlb.com/api/v1/game/{game_pk}/linescore").json()
    df_fb = (
        pd.DataFrame(ls_fb["innings"])[["num","away","home"]]
          .rename(columns={"num":"Inning","away":"Away","home":"Home"})
    )
    df_fb["Away Total"] = ls_fb["teams"]["away"]["runs"]
    df_fb["Home Total"] = ls_fb["teams"]["home"]["runs"]
    show_linescore(df_fb, "Fallback")

    # Boxscore fallback
    box_fb = requests.get(f"https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore").json()

    def lookup_player(pid, side):
        # try top‐level players, then per‐team players
        top = box_fb.get("players", {})
        if f"ID{pid}" in top:
            return top[f"ID{pid}"]
        return box_fb["teams"][side].get("players", {}).get(f"ID{pid}", {})

    # Batting fallback
    bat_fb = []
    for side in ("home","away"):
        team = box_fb["teams"][side]
        for pid in team.get("batters",[]):
            p = lookup_player(pid, side)
            stats = p.get("stats",{}).get("batting",{})
            if stats:
                bat_fb.append({
                    "Team": team["team"]["name"],
                    "Player": p.get("person",{}).get("fullName",""),
                    **stats
                })
    show_stats(bat_fb, "Fallback Batting")

    # Pitching fallback
    pit_fb = []
    for side in ("home","away"):
        team = box_fb["teams"][side]
        for pid in team.get("pitchers",[]):
            p = lookup_player(pid, side)
            stats = p.get("stats",{}).get("pitching",{})
            if stats:
                pit_fb.append({
                    "Team": team["team"]["name"],
                    "Player": p.get("person",{}).get("fullName",""),
                    **stats
                })
    show_stats(pit_fb, "Fallback Pitching")


Live data unavailable—using boxscore & linescore endpoints.

Fallback Linescore


,Inning,Away,Home,Away Total,Home Total
0,1,"{'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBas...","{'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBas...",5,9
1,2,"{'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBas...","{'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBas...",5,9
2,3,"{'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBas...","{'runs': 2, 'hits': 3, 'errors': 0, 'leftOnBas...",5,9
3,4,"{'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBas...","{'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBas...",5,9
4,5,"{'runs': 0, 'hits': 1, 'errors': 1, 'leftOnBas...","{'runs': 4, 'hits': 4, 'errors': 0, 'leftOnBas...",5,9
5,6,"{'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBas...","{'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBas...",5,9
6,7,"{'runs': 1, 'hits': 1, 'errors': 0, 'leftOnBas...","{'runs': 3, 'hits': 4, 'errors': 0, 'leftOnBas...",5,9
7,8,"{'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBas...","{'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBas...",5,9
8,9,"{'runs': 4, 'hits': 5, 'errors': 0, 'leftOnBas...","{'hits': 0, 'errors': 0, 'leftOnBase': 0}",5,9


Fallback Batting


,Team,Player,summary,gamesPlayed,flyOuts,groundOuts,airOuts,runs,doubles,triples,...,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,atBatsPerHomeRun,popOuts,lineOuts,note
0,Kansas City Royals,Jonathan India,"4-5 | 2B, RBI, 3 R",1,1,0,1,3,1,0,...,1,0,0,0,0,0,-.--,0,0,NaN
1,Kansas City Royals,Freddy Fermin,0-0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,-.--,0,0,NaN
2,Kansas City Royals,Bobby Witt Jr.,"4-5 | 2B, 3 R",1,0,1,0,3,1,0,...,0,1,0,0,0,0,-.--,0,0,NaN
3,Kansas City Royals,Maikel Garcia,"1-3 | 2B, BB, 2 RBI",1,0,1,1,0,1,0,...,2,4,0,0,0,0,-.--,0,1,NaN
4,Kansas City Royals,John Rave,0-1,1,1,0,1,0,0,0,...,0,1,0,0,0,0,-.--,0,0,b-
5,Kansas City Royals,Vinnie Pasquantino,"2-4 | HR, 2B, K",1,0,1,0,2,1,0,...,5,3,0,0,0,0,4.00,0,0,NaN
6,Kansas City Royals,Salvador Perez,"1-4 | 2B, RBI",1,0,1,2,0,1,0,...,1,0,0,0,0,0,-.--,1,1,NaN
7,Kansas City Royals,Jac Caglianone,0-4 | 3 K,1,1,0,1,0,0,0,...,0,1,0,0,0,0,-.--,0,0,NaN
8,Kansas City Royals,Nick Loftin,"0-3 | HBP, K",1,0,1,1,0,0,0,...,0,0,0,0,0,0,-.--,1,0,NaN
9,Kansas City Royals,Drew Waters,"1-3 | BB, K, CS",1,0,1,0,0,0,0,...,0,2,0,0,0,0,-.--,0,0,NaN


Fallback Pitching


,Team,Player,note,summary,gamesPlayed,gamesStarted,flyOuts,groundOuts,airOuts,runs,...,runsScoredPer9,homeRunsPer9,inheritedRunners,inheritedRunnersScored,catchersInterference,sacBunts,sacFlies,passedBall,popOuts,lineOuts
0,Kansas City Royals,Seth Lugo,"(W, 5-5)","5.2 IP, 0 ER, 8 K, 5 BB",1,1,6,1,8,0,...,0.00,0.00,0,0,0,0,0,0,1,1
1,Kansas City Royals,Daniel Lynch IV,NaN,"1.1 IP, ER, 2 K, 0 BB",1,0,0,1,1,1,...,6.75,6.75,2,0,0,0,0,0,1,0
2,Kansas City Royals,John Schreiber,NaN,"1.0 IP, 0 ER, 2 K, 0 BB",1,0,0,0,1,0,...,0.00,0.00,0,0,0,0,0,0,0,1
3,Kansas City Royals,Sam Long,NaN,"1.0 IP, 4 ER, 0 K, BB",1,0,1,1,2,4,...,36.00,0.00,0,0,0,0,0,0,1,0
4,Los Angeles Dodgers,Shohei Ohtani,NaN,"2.0 IP, 0 ER, K, BB",1,1,1,1,3,0,...,0.00,0.00,0,0,0,0,0,0,1,1
5,Los Angeles Dodgers,Ben Casparius,"(L, 6-2)","4.0 IP, 6 ER, 4 K, BB",1,0,1,3,4,6,...,13.50,2.25,0,0,0,0,0,0,2,1
6,Los Angeles Dodgers,Luis García,NaN,"0.1 IP, 3 ER, 0 K, 0 BB",1,0,0,1,0,3,...,81.00,0.00,0,0,0,0,0,0,0,0
7,Los Angeles Dodgers,Anthony Banda,NaN,"0.2 IP, 0 ER, K, 0 BB",1,0,1,0,1,0,...,0.00,0.00,1,0,0,0,0,0,0,0
8,Los Angeles Dodgers,Miguel Rojas,NaN,"1.0 IP, 0 ER, 0 K, 0 BB",1,0,1,1,2,0,...,0.00,0.00,0,0,0,0,0,0,0,1
